In [23]:
## This notebook simply get patient 0 positions
import os
from utils import *
from tqdm import tqdm
from multiprocessing import Pool

In [7]:
dir_tracks = '/gpfs/scratch/yur97/compressed_coverage_tracks/'
dir_out = '/gpfs/scratch/yur97/patient_zero_position'
ltrack =  os.listdir(dir_tracks)

In [10]:
ltrack = [os.path.join(dir_tracks,i) for i in ltrack]

In [13]:
def get_zero_position(coverage_file):
    id_aliquot = coverage_file.split('.')[0].split('/')[-1]
    dict_patient = {}
    dict_patient[id_aliquot] = {}
    with open(coverage_file, 'r+b') as wig_f:
        mwig = mmap.mmap(wig_f.fileno(), 0, prot=mmap.PROT_READ)
        itmwig = iter(mwig.readline, b"")
        next(itmwig)
        for lines in itmwig:
            lines = lines.decode("utf-8") 
            if lines.startswith('fixed'):
                line_list = re.findall(r'\d+', lines)
                chr_n = line_list[0]
                start_pos = int(line_list[1])
                position = start_pos - 1
            else:
                position += 1
            if chr_n not in dict_patient[id_aliquot]:
                dict_patient[id_aliquot][chr_n] = []
            if lines == '0\n':
                dict_patient[id_aliquot][chr_n].append(position)
        mwig.close()
    
    pickle.dump(dict_patient, open(os.path.join(dir_out,id_aliquot+'.pkl'),'wb'))

In [26]:
p = Pool(20)
tqdm(p.imap(get_zero_position, ltrack), total=len(ltrack))
p.close()
p.join()



  0%|          | 0/2761 [00:00<?, ?it/s]


KeyboardInterrupt: 

Process ForkPoolWorker-81:
Traceback (most recent call last):
  File "/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/gpfs/group/epo2/default/yur97/gitlab/pcawg-to-mutsigcv/cov_process/run_cov_calc/run_cov_061921/utils.py", line 467, in get_zero_position
    lines = lines.decode("utf-8")
KeyboardInterrupt
